Code to generate samples using the standard Metropolis Algorithm

In [ ]:
using Random, Statistics, Optim, SpecialFunctions

# -----------------------------
# Helper: Periodic Boundary Conditions
# -----------------------------
periodic(i, L) = mod(i - 1, L) + 1

# -----------------------------
# Lattice Initialization
# -----------------------------
"""
    initialize_lattice(L)

Creates an L×L lattice:
  - sigma: Ising spins (±1)
  - U1: Horizontal gauge links (U(1) elements: exp(iθ))
  - U2: Vertical gauge links (U(1) elements)
"""
function initialize_lattice(L)
    sigma = [rand(Bool) ? 1 : -1 for i in 1:L, j in 1:L]
    U1    = [exp(1im * 2π * rand()) for i in 1:L, j in 1:L]
    U2    = [exp(1im * 2π * rand()) for i in 1:L, j in 1:L]
    return sigma, U1, U2
end

# -----------------------------
# Total Energy Computation
# -----------------------------
"""
    total_energy(sigma, U1, U2, K, beta)

Compute the total energy of the configuration.
- Gauge energy: sum over plaquettes,
    where a plaquette at (i,j) is
      P = U1[i,j] * U2[i, j+1] * conj(U1[i+1,j]) * conj(U2[i,j])
  and the contribution is -K·Re(P).
- Matter energy: sum over nearest neighbors (horizontal and vertical),
    where the coupling is -β · (sigma(x)·Re(U(x,y))·sigma(y)).
"""
function total_energy(sigma, U1, U2, K, beta)
    L = size(sigma, 1)
    E = 0.0
    # Gauge part: loop over sites and form plaquettes.
    for i in 1:L, j in 1:L
        jp = periodic(j+1, L)
        ip = periodic(i+1, L)
        P = U1[i,j] * U2[i, jp] * conj(U1[ip, j]) * conj(U2[i,j])
        E += -K * real(P)
    end
    # Matter part: horizontal and vertical nearest neighbors.
    for i in 1:L, j in 1:L
        jp = periodic(j+1, L)
        ip = periodic(i+1, L)
        E += -beta * ( sigma[i,j] * real(U1[i,j]) * sigma[i, jp] +
                       sigma[i,j] * real(U2[i,j]) * sigma[ip, j] )
    end
    return E
end

# -----------------------------
# Metropolis Sweep Update
# -----------------------------
"""
    metropolis_sweep!(sigma, U1, U2, K, beta; eps_U=0.5)

Performs one full sweep over the lattice:
  - For each spin: propose a flip (change sign) and accept using Metropolis rule.
  - For each gauge link (U1 and U2): propose a new link by rotating the current link 
    by δ (with δ uniformly drawn from [-eps_U, eps_U]) and accept with probability exp(-ΔE).
"""
function metropolis_sweep!(sigma, U1, U2, K, beta; eps_U=0.5)
    L = size(sigma,1)
    
    # --- Update spins ---
    for i in 1:L, j in 1:L
        # Compute energy before flipping
        E_old = total_energy(sigma, U1, U2, K, beta)
        sigma[i,j] *= -1  # propose flip
        E_new = total_energy(sigma, U1, U2, K, beta)
        ΔE = E_new - E_old
        if rand() >= exp(-ΔE)
            sigma[i,j] *= -1  # reject: flip back
        end
    end

    # --- Update horizontal gauge links (U1) ---
    for i in 1:L, j in 1:L
        old_link = U1[i,j]
        E_old = total_energy(sigma, U1, U2, K, beta)
        δ = rand()*2*eps_U - eps_U
        U1[i,j] = old_link * exp(1im * δ)  # propose update
        E_new = total_energy(sigma, U1, U2, K, beta)
        ΔE = E_new - E_old
        if rand() >= exp(-ΔE)
            U1[i,j] = old_link  # reject update
        end
    end

    # --- Update vertical gauge links (U2) ---
    for i in 1:L, j in 1:L
        old_link = U2[i,j]
        E_old = total_energy(sigma, U1, U2, K, beta)
        δ = rand()*2*eps_U - eps_U
        U2[i,j] = old_link * exp(1im * δ)
        E_new = total_energy(sigma, U1, U2, K, beta)
        ΔE = E_new - E_old
        if rand() >= exp(-ΔE)
            U2[i,j] = old_link
        end
    end
end

# -----------------------------
# Sample Generation
# -----------------------------
"""
    generate_samples(L, K, beta, n_thermal, n_samples, sweeps_between)

Thermalizes the lattice for n_thermal full sweeps, then collects n_samples 
samples, performing sweeps_between full sweeps between samples.
Each sample is stored (deep copied) along with its energy per site.
"""
function generate_samples(L, K, beta, n_thermal, n_samples, sweeps_between)
    sigma, U1, U2 = initialize_lattice(L)
    # Thermalize
    for s in 1:n_thermal
        metropolis_sweep!(sigma, U1, U2, K, beta)
    end
    samples = Vector{Tuple{Array{Int,2}, Array{ComplexF64,2}, Array{ComplexF64,2}}}(undef, n_samples)
    energies = zeros(n_samples)
    for s in 1:n_samples
        for k in 1:sweeps_between
            metropolis_sweep!(sigma, U1, U2, K, beta)
        end
        samples[s] = (deepcopy(sigma), deepcopy(U1), deepcopy(U2))
        energies[s] = total_energy(sigma, U1, U2, K, beta) / (L^2)
    end
    return samples, energies
end

# -----------------------------
# Staples for Gauge Pseudolikelihood
# -----------------------------

# -----------------------------
# Main: Run Simulation and Parameter Estimation
# -----------------------------
# Simulation parameters
L = 7
K_true = 1.2
beta_true = 0.34
n_thermal = 100         # Number of sweeps for thermalization
n_samples = 10000          # Number of samples to collect
sweeps_between = 10     # Number of full sweeps between samples

# Generate samples from the model.
samples, energies = generate_samples(L, K_true, beta_true, n_thermal, n_samples, sweeps_between)
println("Collected ", n_samples, " samples.")
println("Average energy per site: ", mean(energies))


Now code to learn $\beta$ using Pseudo-likelihood. 
\begin{align*}
    P\left(\sigma_{ij} \mid \text{neighbors}\right) &= \frac{\exp\Bigl(\beta\,\sigma_{ij}\,H_{ij}\Bigr)}{2\cosh\Bigl(\beta\,H_{ij}\Bigr)}\\
    H_{ij} &= \Re(U_{1,i,j-1})\,\sigma_{i,j-1} + \Re(U_{1,i,j})\,\sigma_{i,j+1} + \Re(U_{2,i-1,j})\,\sigma_{i-1,j} + \Re(U_{2,i,j})\,\sigma_{i+1,j}.
\end{align*}
The overall pseudolikelihood for the lattice is given by
\begin{align*}
    \mathcal{PL}(\beta) = \prod_{i,j} \frac{\exp\Bigl(\beta\,\sigma_{ij}\,H_{ij}\Bigr)}
{2\cosh\Bigl(\beta\,H_{ij}\Bigr)}.
\end{align*}

In [ ]:
using Optim, SpecialFunctions

# Compute the log pseudolikelihood for one configuration.
# For each site (i,j), the local effective field is given by:
#   H[i,j] = real(U1[i,j-1])*σ[i,j-1] + real(U1[i,j])*σ[i,j+1]
#          + real(U2[i-1,j])*σ[i-1,j] + real(U2[i,j])*σ[i+1,j]
function log_pseudolikelihood_sample(beta, sigma, U1, U2)
    L = size(sigma, 1)
    lp = 0.0
    for i in 1:L, j in 1:L
        # Apply periodic boundary conditions for neighbors
        left_j  = periodic(j - 1, L)
        right_j = periodic(j + 1, L)
        up_i    = periodic(i - 1, L)
        down_i  = periodic(i + 1, L)

        # Local field contributions from the four neighbors
        H_left  = real(U1[i, left_j])  * sigma[i, left_j]
        H_right = real(U1[i, j])         * sigma[i, right_j]
        H_up    = real(U2[up_i, j])      * sigma[up_i, j]
        H_down  = real(U2[i, j])         * sigma[down_i, j]
        H = H_left + H_right + H_up + H_down

        # Log pseudolikelihood contribution for site (i,j)
        lp += beta * sigma[i,j] * H - log(2 * cosh(beta * H))
    end
    return lp
end

# Total (negative) log pseudolikelihood over all samples.
# We sum the contributions from each sample configuration.
function neg_total_log_pseudolikelihood(beta, samples)
    total_lp = 0.0
    for (sigma, U1, U2) in samples
        total_lp += log_pseudolikelihood_sample(beta, sigma, U1, U2)
    end
    return -total_lp  # negative because we maximize the log pseudolikelihood
end

# Now, use the Optim package to learn beta.
# We'll minimize the negative total log pseudolikelihood.
# Here, we use a search interval [0.0, 2.0]; you can adjust it as needed.
result = optimize(b -> neg_total_log_pseudolikelihood(b, samples), 0.0, 2.0)
beta_est = Optim.minimizer(result)
println("Estimated beta from pseudolikelihood: ", beta_est)

Now the code to learn $K$ values, after fixing $\beta$.

In [ ]:
# --- Hyvärinen Score for U1 and U2 ---
# Here we work in the angular (phase) representation.
# For a gauge link U = exp(i θ), the contribution to the Hyvärinen score is:
#   (1/2)[(dE/dθ)²] - (d²E/dθ²)
# where dE/dθ and d²E/dθ² include contributions from both the matter and the gauge parts.

# Hyvärinen score for U1 links
function hyvarinen_score_U1(sigma, U1, U2, K, beta)
    L = size(sigma, 1)
    score = 0.0
    for i in 1:L, j in 1:L
        # Get the phase of U1[i,j]
        θ = angle(U1[i,j])
        
        # -- Matter contribution (only one term involves U1[i,j]) --
        # Matter term: -β σ[i,j] cos(θ) σ[i,j+1]
        jp = periodic(j+1, L)
        dE_matter = beta * sigma[i,j] * sigma[i,jp] * sin(θ)
        d2E_matter = beta * sigma[i,j] * sigma[i,jp] * cos(θ)
        
        # -- Gauge contributions --
        # U1[i,j] appears in two plaquettes:
        # 1. Plaquette at (i,j): U1 appears directly.
        jp_plaq = periodic(j+1, L)
        ip_plaq = periodic(i+1, L)
        # The corresponding term is: -K * cos(θ + φ₁),
        # where φ₁ = arg( U2[i, jp_plaq] * conj(U1[ip_plaq,j]) * conj(U2[i,j]) )
        A = U2[i, jp_plaq] * conj(U1[ip_plaq,j]) * conj(U2[i,j])
        φ₁ = angle(A)
        dE_gauge1 = K * sin(θ + φ₁)
        d2E_gauge1 = K * cos(θ + φ₁)
        
        # 2. Plaquette at (i-1,j): U1 appears as the complex conjugate.
        im = periodic(i-1, L)
        # For the plaquette at (i-1,j):
        # Term = -K * cos(φ₂ - θ), with φ₂ = arg( U1[im,j] * U2[im, periodic(j+1,L)] * conj(U2[im,j]) )
        jp_im = periodic(j+1, L)
        A2 = U1[im, j] * U2[im, jp_im] * conj(U2[im,j])
        φ₂ = angle(A2)
        # Derivative: d/dθ [ -K cos(φ₂ - θ) ] = -K * sin(φ₂ - θ)
        dE_gauge2 = -K * sin(φ₂ - θ)
        d2E_gauge2 = K * cos(φ₂ - θ)
        
        # Total derivatives at U1[i,j]
        dE_total = dE_matter + dE_gauge1 + dE_gauge2
        d2E_total = d2E_matter + d2E_gauge1 + d2E_gauge2
        
        # Add Hyvärinen score contribution for this link.
        score += 0.5 * dE_total^2 - d2E_total
    end
    return score
end

# Hyvärinen score for U2 links
function hyvarinen_score_U2(sigma, U1, U2, K, beta)
    L = size(sigma, 1)
    score = 0.0
    for i in 1:L, j in 1:L
        # Get the phase of U2[i,j]
        θ = angle(U2[i,j])
        
        # -- Matter contribution (only one term involves U2[i,j]) --
        # Matter term: -β σ[i,j] cos(θ) σ[i+1,j]
        ip = periodic(i+1, L)
        dE_matter = beta * sigma[i,j] * sigma[ip,j] * sin(θ)
        d2E_matter = beta * sigma[i,j] * sigma[ip,j] * cos(θ)
        
        # -- Gauge contributions --
        # U2[i,j] appears in two plaquettes:
        # 1. Plaquette at (i,j): Here U2[i,j] enters as its complex conjugate.
        jp = periodic(j+1, L)
        ip_plaq = periodic(i+1, L)
        # For plaquette (i,j): term = -K * cos(φ₃ - θ),
        # with φ₃ = arg( U1[i,j] * U2[i,jp] * conj(U1[ip,j]) )
        A = U1[i,j] * U2[i, jp] * conj(U1[ip,j])
        φ₃ = angle(A)
        dE_gauge1 = K * sin(φ₃ - θ)
        d2E_gauge1 = K * cos(φ₃ - θ)
        
        # 2. Plaquette at (i, j-1): Here U2[i,j] appears directly.
        jm = periodic(j-1, L)
        # For plaquette (i,j-1): term = -K * cos(θ + φ₄),
        # where φ₄ = arg( U1[i,j-1] * conj(U1[periodic(i+1,L), j-1]) * conj(U2[i,j-1]) )
        B = U1[i, jm] * conj(U1[periodic(i+1,L), jm]) * conj(U2[i, jm])
        φ₄ = angle(B)
        dE_gauge2 = K * sin(θ + φ₄)
        d2E_gauge2 = K * cos(θ + φ₄)
        
        dE_total = dE_matter + dE_gauge1 + dE_gauge2
        d2E_total = d2E_matter + d2E_gauge1 + d2E_gauge2
        
        score += 0.5 * dE_total^2 - d2E_total
    end
    return score
end

function hyvarinen_score_gauge(sigma, U1, U2, K, beta)
    score_U1 = hyvarinen_score_U1(sigma, U1, U2, K, beta)
    score_U2 = hyvarinen_score_U2(sigma, U1, U2, K, beta)
    return score_U1 + score_U2
end

function objective_K(samples::Vector{Tuple{Matrix{Int64}, Matrix{ComplexF64}, Matrix{ComplexF64}}},K::Float64, beta_est::Float64)
    total_score = 0.0
    for (sigma, U1, U2) in samples
        total_score += hyvarinen_score_gauge(sigma, U1, U2, K, beta_est)
    end
    return total_score / length(samples)
end

Now the code to learn $\beta$ and $K$ for various sample sizes.

In [ ]:
using Optim, Statistics, Plots

# --- Parameters ---
L = 4
K_true = 2.5
beta_true = 0.25
n_thermal = 100         # Number of sweeps for thermalization
sweeps_between = 1     # Sweeps between samples

# Define a range of sample sizes to test.
sample_sizes = [4000*2^i for i in 4:10]

# Arrays to store the average absolute errors for beta and K.
beta_errors_avg = Float64[]
K_errors_avg = Float64[]

# Number of independent runs for each sample size.
n_runs = 1

# Loop over different sample sizes.
for n_samples in sample_sizes
    println("Running for n_samples = $n_samples")
    run_beta_errors = Float64[]
    run_K_errors = Float64[]
    
    for run in 1:n_runs
        println("  Run $run")
        # Generate samples for the current sample size.
        samples, energies = generate_samples(L, K_true, beta_true, n_thermal, n_samples, sweeps_between)
        println(eltype(samples))
        # -----------------------------
        # Learn beta using the pseudolikelihood
        # -----------------------------

        # Optimize beta (assume search interval [0,2]; adjust if necessary).
        result_beta = optimize(b -> neg_total_log_pseudolikelihood(b,samples), 0.0, 2.0)
        beta_est = Optim.minimizer(result_beta)
        println("    Estimated beta: ", beta_est)
        
        # -----------------------------
        # Learn K using the Hyvärinen score
        # -----------------------------
        
        # Optimize K (assume search interval [0,10]; adjust if needed).
        result_K = optimize(K -> objective_K(samples,K,beta_est), 0.1, 10.0)
        K_est = Optim.minimizer(result_K)
        println("    Estimated K: ", K_est)
        
        # Record absolute errors for this run.
        push!(run_beta_errors, abs(beta_est - beta_true))
        push!(run_K_errors, abs(K_est - K_true))
    end
    
    # Average the errors over the runs.
    push!(beta_errors_avg, mean(run_beta_errors))
    push!(K_errors_avg, mean(run_K_errors))
end

# -----------------------------
# Plotting the Errors vs. Sample Size
# -----------------------------
p = plot(sample_sizes, beta_errors_avg, lw=2, marker=:circle, xscale=:log10, yscale=:log10,
         xlabel="Number of samples", ylabel="Absolute error", label="β error",
         title="Parameter Estimation Errors vs. Sample Size")
plot!(sample_sizes, K_errors_avg, lw=2, marker=:square, xscale=:log10, yscale=:log10,
      label="K error")
display(p)


In [ ]:
using Plots
# assume `sample_sizes`, `beta_errors_avg`, and `K_errors_avg` already defined

# restrict to the range you are plotting (starts at index 5)
Ns_beta   = sample_sizes
err_beta  = beta_errors_avg
Ns_K      = sample_sizes
err_K     = K_errors_avg

# ---- 1/√N reference curves (anchored at first data point) ----
ref_beta = err_beta[1] .* sqrt.(Ns_beta[1] ./ Ns_beta)  # passes through (Ns_beta[1], err_beta[1])
ref_K    = err_K[1]    .* sqrt.(Ns_K[1]    ./ Ns_K)     # passes through (Ns_K[1],   err_K[1])

# --------------------- β error plot ---------------------------
p_beta = plot(sample_sizes, err_beta;
    lw      = 2,
    marker  = :o,
    xscale  = :log2,
    yscale  = :log2,
    palette = :RdBu_4,
    legend  = false)

plot!(p_beta, sample_sizes, ref_beta;
    lw       = 1.5,
    ls       = :dash,
    color    = :black)            # 1/√N guide

savefig(p_beta, "beta_errors.pdf")
display(p_beta)

# --------------------- K error plot ---------------------------
p_K = plot(sample_sizes, err_K;
    lw      = 2,
    marker  = :o,
    xscale  = :log10,
    yscale  = :log10,
    xlabel  = "Number of samples",
    ylabel  = "Absolute error",
    title   = "K Parameter Estimation Error vs. Sample Size",
    color   = my_blue,
    legend  = false)

savefig(p_K, "K_errors.pdf")
display(p_K)

We implement an RG scheme here - we use a mix of Kadanoff's RG and mapping gauge-links to gauge-links.

In [ ]:
"""
    coarse_grain_overlapping(sigma, U1, U2; block_size=3)

Perform an overlapping block coarse–graining transformation on the lattice.
Each block is of size block_size × block_size (default 3×3) and is defined 
for every starting position in the original lattice. The new (coarse–grained)
lattice has size (L - block_size + 1) × (L - block_size + 1).

For each block starting at (i, j):
  - The coarse–grained horizontal link is the product of the horizontal links
    along the top row of the block:
      U1_new(i,j) = ∏ₖ U1(i, j+k-1), k = 1,...,block_size.
  - The coarse–grained vertical link is the product of the vertical links along
    the left column of the block:
      U2_new(i,j) = ∏ₖ U2(i+k-1, j), k = 1,...,block_size.
  - The coarse–grained spin is the product of the spins in the left column of the block:
      sigma_new(i,j) = ∏ₖ sigma(i+k-1, j), k = 1,...,block_size.

Returns the coarse–grained configuration (sigma_new, U1_new, U2_new).
"""
function coarse_grain_overlapping(sigma, U1, U2; block_size=2)
    L = size(sigma, 1)
    L_new = div(L + 1,2)  # new lattice size (overlapping blocks)
    
    sigma_new = Array{Int}(undef, L_new, L_new)
    U1_new = Array{ComplexF64}(undef, L_new, L_new)
    U2_new = Array{ComplexF64}(undef, L_new, L_new)
    
    for i in 1:L_new, j in 1:L_new
        # Coarse–grained spin: product over the left column of the block.
        s = sigma[mod1(2i-1,L),mod1(2j-1,L)] + sigma[mod1(2i,L),mod1(2j-1,L)] + sigma[mod1(2i-1,L),mod1(2j,L)] + sigma[mod1(2i,L),mod1(2j,L)]
        if s > 0
            sigma_new[mod1(i,L_new),mod1(j,L_new)] = 1
        elseif s < 0
            sigma_new[mod1(i,L_new),mod1(j,L_new)] = -1
        else
            sigma_new[mod1(i,L_new),mod1(j,L_new)] = rand(Bool) ? 1 : -1  # Random tie-breaker
        end
        
        # Coarse–grained horizontal link: product over the top row of the block.
    
        U1_new[mod1(i,L_new),mod1(j,L_new)] = U1[mod1(2i-1,L),mod1(2j-1,L)]*U1[mod1(2i-1,L),mod1(2j,L)]
        
        # Coarse–grained vertical link: product over the left column of the block.
        U2_new[mod1(i,L_new),mod1(j,L_new)] = U2[mod1(2i-1,L),mod1(2j-1,L)]*U2[mod1(2i,L),mod1(2j-1,L)]
    end
    return sigma_new, U1_new, U2_new
end


Now implement a full RG procedure!

In [ ]:
# First, ensure you have these wrappers that accept a vector of samples:
function neg_total_log_pseudolikelihood(samples::Vector{Tuple{Matrix{Int}, Matrix{ComplexF64}, Matrix{ComplexF64}}}, beta::Float64)
    total_lp = 0.0
    for (sigma, U1, U2) in samples
        total_lp += log_pseudolikelihood_sample(beta, sigma, U1, U2)
    end
    return -total_lp  # negative because we maximize the log pseudolikelihood
end

function objective_K(samples::Vector{Tuple{Matrix{Int}, Matrix{ComplexF64}, Matrix{ComplexF64}}}, K::Float64, beta::Float64)
    total_score = 0.0
    for (sigma, U1, U2) in samples
        total_score += hyvarinen_score_gauge(sigma, U1, U2, K, beta)
    end
    return total_score / length(samples)
end

# Then update your RG implementation:
function RG_implementation(samples, rg_steps)
    samples_init = samples
    betas = [beta_true]
    Ks = [K_true]
    for i in 1:rg_steps
        # Coarse grain the current samples
        new_samples = [coarse_grain_overlapping(sigma, U1, U2; block_size=2) for (sigma, U1, U2) in samples_init]
        # Estimate beta using the coarse–grained samples
        result_beta = optimize(b -> neg_total_log_pseudolikelihood(new_samples, b), -10.0, 12.0)
        beta_est = Optim.minimizer(result_beta)
        # Estimate K using the coarse–grained samples and estimated beta
        result_K = optimize(K -> objective_K(new_samples, K, beta_est), 0.0, 10.0)
        K_est = Optim.minimizer(result_K)
        push!(betas, beta_est)
        push!(Ks, K_est)
        samples_init = new_samples
    end
    return betas, Ks
end


Code to learn complexity of $\beta$ - the number of samples needed to have mean absolute errors under a certain value.

In [ ]:
using StatsBase, Plots, Random, Statistics, Optim

# -----------------------------
# Parameters for the Simulation and Complexity Study
# -----------------------------

# Simulation parameters (for the gauge model)
L = 4                         # Lattice size (L x L)
K_true = 1.2                   # Fixed gauge coupling (example value)
beta_values = 1.0:0.05:2.5        # Range of "true" β values to test
n_thermal = 100                # Number of sweeps for thermalization
sweeps_between = 10            # Number of full sweeps between samples

# Complexity study parameters for estimating β via pseudolikelihood
trial_count = 50               # Number of independent trials per β at a fixed sample count
initial_sample_count = 200     # Initial number of samples (per trial)
max_samples = 25000            # Maximum allowed number of samples per trial
target_error = 0.01            # Target standard error for estimated β across trials
increase_factor = 1.1          # Multiplicative factor for sample size if error is too high

# Container for recording required sample counts for each β
samples_required = Float64[]

# -----------------------------
# Complexity Study Loop
# -----------------------------
for beta in beta_values
    println("Testing true β = $beta")
    current_sample_count = initial_sample_count
    converged = false

    while !converged && current_sample_count <= max_samples
        trial_beta_estimates = Float64[]
        
        # Run a number of independent trials at the current sample count.
        for trial in 1:trial_count
            println("  Trial $trial with n_samples = $current_sample_count")
            # Generate samples from the model (replace with your actual function).
            # This returns a tuple (samples, energies).
            samples, energies = generate_samples(L, K_true, beta, n_thermal, current_sample_count, sweeps_between)
            
            # Estimate β using your pseudolikelihood function.
            # The optimizer finds b in [0.0, 2.0] minimizing neg_total_log_pseudolikelihood.
            result_beta = optimize(b -> neg_total_log_pseudolikelihood(b,samples), 0.0, 2.0)
            beta_est = Optim.minimizer(result_beta)
            push!(trial_beta_estimates, beta_est)
            
            println("    Estimated β = $(beta_est)")
        end
        
        # Compute the standard error of the β estimates from the trials.
        sem_beta = std(trial_beta_estimates) / sqrt(trial_count)
        println("For true β = $beta with n_samples = $current_sample_count, SEM(β) = $sem_beta")
        
        # Check if the standard error is below threshold.
        if sem_beta <= target_error
            converged = true
            println("Converged for true β = $beta with n_samples = $current_sample_count")
            push!(samples_required, current_sample_count)
        else
            current_sample_count = round(Int, current_sample_count * increase_factor)
            println("Increasing sample count to $current_sample_count for true β = $beta")
        end
    end
    
    # If we never achieved the target error, record NaN.
    if !converged
        println("Did not converge for true β = $beta within maximum samples.")
        push!(samples_required, NaN)
    end
end

# -----------------------------
# Plotting: Beta vs Required Sample Size
# -----------------------------
plot(beta_values, samples_required, marker=:circle, lw=2,
     xlabel="True β", ylabel="Required Sample Size",
     title="Complexity: Required Sample Size vs True β\n(Target SEM ≤ $(target_error))",
     legend=false)


Code for measuring complexity of $K$ values

In [ ]:
using StatsBase, Plots, Random, Statistics, Optim

# --- Helper function to aggregate over samples, if needed ---
# (Assuming objective_K(sample, K, sK) is defined to return a real number from one sample.)
function total_objective_K(samples::Vector{T}, K::Float64, sK::Float64) where T
    total = 0.0
    for sample in samples
        total += objective_K(sample, K, sK)
    end
    return total / length(samples)
end

# -----------------------------
# Parameters for the Simulation and Complexity Study (for K)
# -----------------------------
L = 4                         # Lattice size (L x L)
beta_fixed = 1.25             # Fixed beta value used in objective_K (this is sK)
K_values = 0.5:0.1:2.5         # Range of "true" K values to test
n_thermal = 100               # Number of sweeps for thermalization
sweeps_between = 10           # Number of full sweeps between samples

# Complexity study parameters for estimating K via Hyvärinen score
trial_count = 50              # Number of independent trials per K value (for confidence interval)
initial_sample_count = 200    # Initial number of samples (per trial)
max_samples = 25000           # Maximum allowed number of samples per trial
target_error = 0.01           # Target 95% CI half-width for estimated K (i.e. we require: 1.96 * SEM ≤ target_error)
increase_factor = 1.2         # Multiplicative factor for increasing sample count

# Container to record the required sample count for each true K value.
samples_required_K = Float64[]

# -----------------------------
# Complexity Study Loop for K
# -----------------------------
for K_true in K_values
    println("Testing true K = $K_true")
    current_sample_count = initial_sample_count
    converged = false

    while current_sample_count ≤ max_samples && !converged
        trial_K_estimates = Float64[]
        
        # Run trial_count independent trials at the current sample count.
        for trial in 1:trial_count
            println("  Trial $trial with n_samples = $current_sample_count")
            # Generate samples from the model.
            # Assume generate_samples(L, K, beta, n_thermal, n_samples, sweeps_between)
            # returns a vector of sample configurations and associated energies.
            samples, energies = generate_samples(L, K_true, beta_fixed, n_thermal, round(Int, current_sample_count), sweeps_between)
            
            # Estimate K using your pseudolikelihood-based objective.
            # (Using a simple univariate optimization over K in, say, [0, 10].)
            result_K = optimize(K -> objective_K(samples, K, beta_fixed), 0.0, 10.0)
            K_est = Optim.minimizer(result_K)
            push!(trial_K_estimates, K_est)
            println("    Estimated K = $(K_est)")
        end
        
        # Compute the standard error (SEM) and the 95% CI half-width.
        sem_K = std(trial_K_estimates) / sqrt(trial_count)
        ci_half_width = 1.96 * sem_K
        println("For true K = $K_true with n_samples = $(current_sample_count), 95% CI half-width = $(ci_half_width)")
        
        if ci_half_width ≤ target_error
            println("  ✓ Converged for true K = $K_true with n_samples = $(current_sample_count)")
            push!(samples_required_K, round(Int, current_sample_count))
            converged = true
        else
            current_sample_count = round(Int, current_sample_count * increase_factor)
            println("Increasing sample count to $(current_sample_count) for true K = $K_true")
        end
    end

    # If we never reached the target CI half-width, record NaN.
    if !converged
        println("  ✗ True K = $K_true: Did not reach target CI half-width")
        push!(samples_required_K, NaN)
    end
end

# -----------------------------
# Plotting: True K vs. Required Sample Size
# -----------------------------
p = plot(K_values, samples_required_K, marker=:circle, lw=2,
         xlabel="True K", ylabel="Required Sample Size",
         title="Complexity: Required Sample Size vs True K\n(95% CI half-width ≤ $(target_error))",
         legend=false)
display(p)


Now a code to run a full RG trajectory!

In [ ]:
function run_RG_trajectory(K0, β0; L=16, n_samples=10_000,
                           n_thermal=200, sweeps_between=10,
                           rg_steps=3)

    # ---- generate Monte-Carlo data at (K0, β0) ----
    samples, _ = generate_samples(L, K0, β0,
                                  n_thermal, n_samples, sweeps_between)

    # make K₀, β₀ visible inside RG_implementation
    global K_true  = K0
    global beta_true = β0

    βs, Ks = RG_implementation(samples, rg_steps)
    return βs, Ks
end

In [ ]:
K_fixed   = 0.1
β0_list   = 1.0:0.2:2.4
L = 8
rg_steps=1
n_samples=10_000
n_thermal = 2000
sweeps_between=8

for (idx, β0) in enumerate(β0_list)
    βs, Ks = run_RG_trajectory(K_fixed, β0;
                                L=L, n_samples=n_samples, rg_steps=rg_steps,
                                n_thermal=n_thermal, sweeps_between=sweeps_between)
    println("Initial β₀ = $β0: Final (β, K) = ($(βs[end]), $(Ks[end]))")
end
